In [8]:
#Example 1: 

import numpy as np
def are_axes_equivalent(axis2, axis1, angle_threshold=2.0):
    """
    Check if two axes are equivalent considering crystal symmetry and slight deviations by comparing their misorientation angle.
    
    Parameters:
    axis1, axis2 (numpy arrays): The axes to compare
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether the axes are equivalent
    """
    # Normalize both axes
    axis1 = axis1 / np.linalg.norm(axis1)
    axis2 = axis2 / np.linalg.norm(axis2)
    
    # Generate all permutations with sign changes of axis1 to account for Cubic symmetry
    axis_permutations = [
        [ axis1[0],  axis1[1],  axis1[2]],
        [-axis1[0],  axis1[1],  axis1[2]],
        [ axis1[0], -axis1[1],  axis1[2]],
        [ axis1[0],  axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1],  axis1[2]],
        [-axis1[0],  axis1[1], -axis1[2]],
        [ axis1[0], -axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1], -axis1[2]],
        [ axis1[1],  axis1[0],  axis1[2]],
        [-axis1[1],  axis1[0],  axis1[2]],
        [ axis1[1], -axis1[0],  axis1[2]],
        [ axis1[1],  axis1[0], -axis1[2]],
        [-axis1[1], -axis1[0],  axis1[2]],
        [-axis1[1],  axis1[0], -axis1[2]],
        [ axis1[1], -axis1[0], -axis1[2]],
        [-axis1[1], -axis1[0], -axis1[2]],
        [ axis1[2],  axis1[1],  axis1[0]],
        [-axis1[2],  axis1[1],  axis1[0]],
        [ axis1[2], -axis1[1],  axis1[0]],
        [ axis1[2],  axis1[1], -axis1[0]],
        [-axis1[2], -axis1[1],  axis1[0]],
        [-axis1[2],  axis1[1], -axis1[0]],
        [ axis1[2], -axis1[1], -axis1[0]],
        [-axis1[2], -axis1[1], -axis1[0]],
        [ axis1[0],  axis1[2],  axis1[1]],
        [-axis1[0],  axis1[2],  axis1[1]],
        [ axis1[0], -axis1[2],  axis1[1]],
        [ axis1[0],  axis1[2], -axis1[1]],
        [-axis1[0], -axis1[2],  axis1[1]],
        [-axis1[0],  axis1[2], -axis1[1]],
        [ axis1[0], -axis1[2], -axis1[1]],
        [-axis1[0], -axis1[2], -axis1[1]],
        [ axis1[1],  axis1[2],  axis1[0]],
        [-axis1[1],  axis1[2],  axis1[0]],
        [ axis1[1], -axis1[2],  axis1[0]],
        [ axis1[1],  axis1[2], -axis1[0]],
        [-axis1[1], -axis1[2],  axis1[0]],
        [-axis1[1],  axis1[2], -axis1[0]],
        [ axis1[1], -axis1[2], -axis1[0]],
        [-axis1[1], -axis1[2], -axis1[0]],
        [ axis1[2],  axis1[0],  axis1[1]],
        [-axis1[2],  axis1[0],  axis1[1]],
        [ axis1[2], -axis1[0],  axis1[1]],
        [ axis1[2],  axis1[0], -axis1[1]],
        [-axis1[2], -axis1[0],  axis1[1]],
        [-axis1[2],  axis1[0], -axis1[1]],
        [ axis1[2], -axis1[0], -axis1[1]],
        [-axis1[2], -axis1[0], -axis1[1]],
    ]
    
    # Check if axis2 is equivalent to any of the permutations of axis1 within the specified angle threshold
    for perm in axis_permutations:
        # Calculate the cosine of the angle between the two vectors
        cos_angle = np.dot(perm, axis2)
        
        # Ensure the value is within the valid range for arccos
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        
        # Calculate the angle in degrees
        angle = np.degrees(np.arccos(cos_angle))

        
        # Check if the angle is less than the threshold
        if angle <= angle_threshold or (180 - angle) <= angle_threshold:
            print(f"Angle between Axis and CSL Axis {perm:}: {angle:.2f}°")
            return True
    
    return False

def is_sigma_boundary(axis, misorientation_angle, structure_type='FCC', angle_threshold=10.0):
    """
    Determine if the given rotation axis and misorientation angle correspond to a Σ boundary,
    considering the Brandon criterion for deviation and axis symmetry.
    
    Parameters:
    axis (list or tuple): Rotation axis (e.g., [1, 1, 1])
    misorientation_angle (float): Misorientation angle (in degrees)
    structure_type (str): Crystal structure type ('FCC' or 'BCC')
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether it is a Σ boundary
    str: Corresponding Σ value (if it is a Σ boundary)
    float: Actual deviation (in degrees)
    """
    # Predefined Σ values with corresponding rotation axes and misorientation angles for both FCC and BCC
    sigma_data = {
        '3': {'FCC': [{'axis': [1, 1, 1], 'angle': 60}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 60}]},
        '5': {'FCC': [{'axis': [1, 0, 0], 'angle': 36.87}],
              'BCC': [{'axis': [1, 0, 0], 'angle': 36.87}]},
        '7': {'FCC': [{'axis': [1, 1, 1], 'angle': 38.21}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 38.21}]},
        '9': {'FCC': [{'axis': [1, 1, 0], 'angle': 38.94}],
              'BCC': [{'axis': [1, 1, 0], 'angle': 38.94}]},
        '11': {'FCC': [{'axis': [1, 1, 0], 'angle': 50.48}],
               'BCC': [{'axis': [1, 1, 0], 'angle': 50.48}]},
        '13a': {'FCC': [{'axis': [1, 0, 0], 'angle': 22.62}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 22.62}]},
        '13b': {'FCC': [{'axis': [1, 1, 1], 'angle': 27.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 27.79}]},
        '15': {'FCC': [{'axis': [2, 1, 0], 'angle': 48.2}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 48.2}]},        
        '17a': {'FCC': [{'axis': [1, 0, 0], 'angle': 28.07}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 28.07}]},
        '17b': {'FCC': [{'axis': [2, 2, 1], 'angle': 61.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 61.9}]},
        '19a': {'FCC': [{'axis': [1, 1, 0], 'angle': 26.53}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 26.53}]},
        '19b': {'FCC': [{'axis': [1, 1, 1], 'angle': 46.8}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 46.8}]},
        '21a': {'FCC': [{'axis': [1, 1, 1], 'angle': 21.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 21.79}]},
        '21b': {'FCC': [{'axis': [2, 1, 1], 'angle': 44.4}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 44.4}]},
        '23': {'FCC': [{'axis': [3, 1, 1], 'angle': 40.5}],
               'BCC': [{'axis': [3, 1, 1], 'angle': 40.5}]},
        '25a': {'FCC': [{'axis': [1, 0, 0], 'angle': 16.3}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 16.3}]},
        '25b': {'FCC': [{'axis': [3, 3, 1], 'angle': 51.7}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 51.7}]},
        '27a': {'FCC': [{'axis': [1, 1, 0], 'angle': 31.59}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 31.59}]},
        '27b': {'FCC': [{'axis': [2, 1, 0], 'angle': 35.43}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 35.43}]},
        '29a': {'FCC': [{'axis': [1, 0, 0], 'angle': 43.6}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 43.6}]},
        '29b': {'FCC': [{'axis': [2, 2, 1], 'angle': 46.4}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 46.4}]},
        '31a': {'FCC': [{'axis': [1, 1, 1], 'angle': 17.9}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 17.9}]},
        '31b': {'FCC': [{'axis': [2, 1, 1], 'angle': 52.2}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 52.2}]},
        '33a': {'FCC': [{'axis': [1, 1, 0], 'angle': 20.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 20.0}]},
        '33b': {'FCC': [{'axis': [3, 1, 1], 'angle': 33.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 33.6}]},
        '33c': {'FCC': [{'axis': [1, 1, 0], 'angle': 59.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 59.0}]},
        '35a': {'FCC': [{'axis': [2, 1, 1], 'angle': 34.0}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 34.0}]},
        '35b': {'FCC': [{'axis': [3, 3, 1], 'angle': 43.2}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 43.2}]},
        '37a': {'FCC': [{'axis': [1, 0, 0], 'angle': 18.9}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 18.9}]},
        '37b': {'FCC': [{'axis': [3, 1, 0], 'angle': 43.1}],
                'BCC': [{'axis': [3, 1, 0], 'angle': 43.1}]},
        '37c': {'FCC': [{'axis': [1, 1, 1], 'angle': 50.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 50.6}]},
        '39a': {'FCC': [{'axis': [1, 1, 1], 'angle': 32.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 32.2}]},
        '39b': {'FCC': [{'axis': [3, 2, 1], 'angle': 50.1}],
                'BCC': [{'axis': [3, 2, 1], 'angle': 50.1}]},
        '41a': {'FCC': [{'axis': [1, 0, 0], 'angle': 12.7}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 12.7}]},
        '41b': {'FCC': [{'axis': [2, 1, 0], 'angle': 40.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 40.9}]},
        '41c': {'FCC': [{'axis': [1, 1, 0], 'angle': 55.9}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 55.9}]},
        '43a': {'FCC': [{'axis': [1, 1, 1], 'angle': 15.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 15.2}]},
        '43b': {'FCC': [{'axis': [2, 1, 0], 'angle': 27.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 27.9}]},
        '43c': {'FCC': [{'axis': [3, 3, 2], 'angle': 60.8}],
                'BCC': [{'axis': [3, 3, 2], 'angle': 60.8}]},
        '45a': {'FCC': [{'axis': [3, 1, 1], 'angle': 28.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 28.6}]},
        '45b': {'FCC': [{'axis': [2, 2, 1], 'angle': 36.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 36.9}]},
        '45c': {'FCC': [{'axis': [2, 2, 1], 'angle': 53.1}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 53.1}]},
        '47a': {'FCC': [{'axis': [3, 3, 1], 'angle': 37.1}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 37.1}]},
        '47b': {'FCC': [{'axis': [3, 2, 0], 'angle': 43.7}],
                'BCC': [{'axis': [3, 2, 0], 'angle': 43.7}]},
        '49a': {'FCC': [{'axis': [1, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 43.6}]},
        '49b': {'FCC': [{'axis': [5, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [5, 1, 1], 'angle': 43.6}]},
        '49c': {'FCC': [{'axis': [3, 2, 2], 'angle': 49.2}],
                'BCC': [{'axis': [3, 2, 2], 'angle': 49.2}]}
    }
    
    # Normalize the given rotation axis
    axis = np.array(axis)
    
    for sigma, configs in sigma_data.items():
        if structure_type not in configs:
            continue
        for config in configs[structure_type]:
            predefined_axis = np.array(config['axis'])
            
            # Check if the given axis is equivalent to the predefined axis
            if are_axes_equivalent(axis, predefined_axis, angle_threshold):
                predefined_angle = config['angle']
                
                # Calculate the deviation
                deviation = abs(misorientation_angle - predefined_angle)
                
                # Calculate the Brandon criterion threshold
                brandon_threshold = 15 / np.sqrt(float(sigma.rstrip('abc')))
                
                # Check if the misorientation angle matches within the Brandon criterion threshold
                if deviation <= brandon_threshold:
                    return True, sigma, deviation
    
    return False, None, None

# Example usage
axis1 = [17, 17, -16]
misorientation_angle1 = 59.6
structure_type = 'FCC'  # 'FCC' or 'BCC'

is_sigma1, sigma_value1, deviation1 = is_sigma_boundary(axis1, misorientation_angle1, structure_type)
if is_sigma1:
    print(f"This is a Σ{sigma_value1} boundary with a deviation of {deviation1:.2f} degrees")
else:
    print("This is not a Σ boundary")

axis2 = [16, 16, -16]
misorientation_angle2 = 60

is_sigma2, sigma_value2, deviation2 = is_sigma_boundary(axis2, misorientation_angle2, structure_type)
if is_sigma2:
    print(f"This is a Σ{sigma_value2} boundary with a deviation of {deviation2:.2f} degrees")
else:
    print("This is not a Σ boundary")

    
axis3 = [0.1142, 0.0523, 0.9921]
misorientation_angle3 = 19.24199995
is_sigma3, sigma_value3, deviation3 = is_sigma_boundary(axis3, misorientation_angle3, structure_type)
if is_sigma3:
    print(f"This is a Σ{sigma_value3} boundary with a deviation of {deviation3:.2f} degrees")
else:
    print("This is not a Σ boundary")
    


Angle between Axis and CSL Axis [0.5773502691896258, 0.5773502691896258, -0.5773502691896258]: 1.62°
This is a Σ3 boundary with a deviation of 0.40 degrees
Angle between Axis and CSL Axis [0.5773502691896258, 0.5773502691896258, -0.5773502691896258]: 0.00°
This is a Σ3 boundary with a deviation of 0.00 degrees
Angle between Axis and CSL Axis [0.0, 0.0, 1.0]: 7.22°
Angle between Axis and CSL Axis [0.0, 0.0, 1.0]: 7.22°
This is a Σ13a boundary with a deviation of 3.38 degrees


In [12]:
#Example 2: 

import numpy as np

def are_axes_equivalent(axis2, axis1, angle_threshold=2.0):
    """
    Check if two axes are equivalent considering crystal symmetry and slight deviations by comparing their misorientation angle.
    
    Parameters:
    axis1, axis2 (numpy arrays): The axes to compare
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether the axes are equivalent
    """
    # Normalize both axes
    axis1 = axis1 / np.linalg.norm(axis1)
    axis2 = axis2 / np.linalg.norm(axis2)
    
    # Generate all permutations with sign changes of axis1 to account for symmetry of Cubic system
    axis_permutations = [
        [ axis1[0],  axis1[1],  axis1[2]],
        [-axis1[0],  axis1[1],  axis1[2]],
        [ axis1[0], -axis1[1],  axis1[2]],
        [ axis1[0],  axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1],  axis1[2]],
        [-axis1[0],  axis1[1], -axis1[2]],
        [ axis1[0], -axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1], -axis1[2]],
        [ axis1[1],  axis1[0],  axis1[2]],
        [-axis1[1],  axis1[0],  axis1[2]],
        [ axis1[1], -axis1[0],  axis1[2]],
        [ axis1[1],  axis1[0], -axis1[2]],
        [-axis1[1], -axis1[0],  axis1[2]],
        [-axis1[1],  axis1[0], -axis1[2]],
        [ axis1[1], -axis1[0], -axis1[2]],
        [-axis1[1], -axis1[0], -axis1[2]],
        [ axis1[2],  axis1[1],  axis1[0]],
        [-axis1[2],  axis1[1],  axis1[0]],
        [ axis1[2], -axis1[1],  axis1[0]],
        [ axis1[2],  axis1[1], -axis1[0]],
        [-axis1[2], -axis1[1],  axis1[0]],
        [-axis1[2],  axis1[1], -axis1[0]],
        [ axis1[2], -axis1[1], -axis1[0]],
        [-axis1[2], -axis1[1], -axis1[0]],
        [ axis1[0],  axis1[2],  axis1[1]],
        [-axis1[0],  axis1[2],  axis1[1]],
        [ axis1[0], -axis1[2],  axis1[1]],
        [ axis1[0],  axis1[2], -axis1[1]],
        [-axis1[0], -axis1[2],  axis1[1]],
        [-axis1[0],  axis1[2], -axis1[1]],
        [ axis1[0], -axis1[2], -axis1[1]],
        [-axis1[0], -axis1[2], -axis1[1]],
        [ axis1[1],  axis1[2],  axis1[0]],
        [-axis1[1],  axis1[2],  axis1[0]],
        [ axis1[1], -axis1[2],  axis1[0]],
        [ axis1[1],  axis1[2], -axis1[0]],
        [-axis1[1], -axis1[2],  axis1[0]],
        [-axis1[1],  axis1[2], -axis1[0]],
        [ axis1[1], -axis1[2], -axis1[0]],
        [-axis1[1], -axis1[2], -axis1[0]],
        [ axis1[2],  axis1[0],  axis1[1]],
        [-axis1[2],  axis1[0],  axis1[1]],
        [ axis1[2], -axis1[0],  axis1[1]],
        [ axis1[2],  axis1[0], -axis1[1]],
        [-axis1[2], -axis1[0],  axis1[1]],
        [-axis1[2],  axis1[0], -axis1[1]],
        [ axis1[2], -axis1[0], -axis1[1]],
        [-axis1[2], -axis1[0], -axis1[1]],
    ]
    
    
    # Check if axis2 is equivalent to any of the permutations of axis1 within the specified angle threshold
    for perm in axis_permutations:
        # Calculate the cosine of the angle between the two vectors
        cos_angle = np.dot(perm, axis2)
        
        # Ensure the value is within the valid range for arccos
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        
        # Calculate the angle in degrees
        angle = np.degrees(np.arccos(cos_angle))

        
        # Check if the angle is less than the threshold
        if angle <= angle_threshold or (180 - angle) <= angle_threshold:
            #print(f"Angle between Axis and CSL Axis {perm}: {angle:.2f}°")
            return True
    
    return False

def is_sigma_boundary(axis, misorientation_angle, structure_type='FCC', angle_threshold=5.0):
    """
    Determine if the given rotation axis and misorientation angle correspond to a Σ boundary,
    considering the Brandon criterion for deviation and axis symmetry.
    
    Parameters:
    axis (list or tuple): Rotation axis (e.g., [1, 1, 1])
    misorientation_angle (float): Misorientation angle (in degrees)
    structure_type (str): Crystal structure type ('FCC' or 'BCC')
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether it is a Σ boundary
    str: Corresponding Σ value (if it is a Σ boundary)
    float: Actual deviation (in degrees)
    """
    # Predefined Σ values with corresponding rotation axes and misorientation angles for both FCC and BCC
    sigma_data = {
        '3': {'FCC': [{'axis': [1, 1, 1], 'angle': 60}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 60}]},
        '5': {'FCC': [{'axis': [1, 0, 0], 'angle': 36.87}],
              'BCC': [{'axis': [1, 0, 0], 'angle': 36.87}]},
        '7': {'FCC': [{'axis': [1, 1, 1], 'angle': 38.21}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 38.21}]},
        '9': {'FCC': [{'axis': [1, 1, 0], 'angle': 38.94}],
              'BCC': [{'axis': [1, 1, 0], 'angle': 38.94}]},
        '11': {'FCC': [{'axis': [1, 1, 0], 'angle': 50.48}],
               'BCC': [{'axis': [1, 1, 0], 'angle': 50.48}]},
        '13a': {'FCC': [{'axis': [1, 0, 0], 'angle': 22.62}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 22.62}]},
        '13b': {'FCC': [{'axis': [1, 1, 1], 'angle': 27.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 27.79}]},
        '15': {'FCC': [{'axis': [2, 1, 0], 'angle': 48.2}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 48.2}]},        
        '17a': {'FCC': [{'axis': [1, 0, 0], 'angle': 28.07}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 28.07}]},
        '17b': {'FCC': [{'axis': [2, 2, 1], 'angle': 61.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 61.9}]},
        '19a': {'FCC': [{'axis': [1, 1, 0], 'angle': 26.53}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 26.53}]},
        '19b': {'FCC': [{'axis': [1, 1, 1], 'angle': 46.8}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 46.8}]},
        '21a': {'FCC': [{'axis': [1, 1, 1], 'angle': 21.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 21.79}]},
        '21b': {'FCC': [{'axis': [2, 1, 1], 'angle': 44.4}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 44.4}]},
        '23': {'FCC': [{'axis': [3, 1, 1], 'angle': 40.5}],
               'BCC': [{'axis': [3, 1, 1], 'angle': 40.5}]},
        '25a': {'FCC': [{'axis': [1, 0, 0], 'angle': 16.3}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 16.3}]},
        '25b': {'FCC': [{'axis': [3, 3, 1], 'angle': 51.7}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 51.7}]},
        '27a': {'FCC': [{'axis': [1, 1, 0], 'angle': 31.59}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 31.59}]},
        '27b': {'FCC': [{'axis': [2, 1, 0], 'angle': 35.43}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 35.43}]},
        '29a': {'FCC': [{'axis': [1, 0, 0], 'angle': 43.6}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 43.6}]},
        '29b': {'FCC': [{'axis': [2, 2, 1], 'angle': 46.4}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 46.4}]},
        '31a': {'FCC': [{'axis': [1, 1, 1], 'angle': 17.9}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 17.9}]},
        '31b': {'FCC': [{'axis': [2, 1, 1], 'angle': 52.2}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 52.2}]},
        '33a': {'FCC': [{'axis': [1, 1, 0], 'angle': 20.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 20.0}]},
        '33b': {'FCC': [{'axis': [3, 1, 1], 'angle': 33.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 33.6}]},
        '33c': {'FCC': [{'axis': [1, 1, 0], 'angle': 59.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 59.0}]},
        '35a': {'FCC': [{'axis': [2, 1, 1], 'angle': 34.0}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 34.0}]},
        '35b': {'FCC': [{'axis': [3, 3, 1], 'angle': 43.2}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 43.2}]},
        '37a': {'FCC': [{'axis': [1, 0, 0], 'angle': 18.9}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 18.9}]},
        '37b': {'FCC': [{'axis': [3, 1, 0], 'angle': 43.1}],
                'BCC': [{'axis': [3, 1, 0], 'angle': 43.1}]},
        '37c': {'FCC': [{'axis': [1, 1, 1], 'angle': 50.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 50.6}]},
        '39a': {'FCC': [{'axis': [1, 1, 1], 'angle': 32.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 32.2}]},
        '39b': {'FCC': [{'axis': [3, 2, 1], 'angle': 50.1}],
                'BCC': [{'axis': [3, 2, 1], 'angle': 50.1}]},
        '41a': {'FCC': [{'axis': [1, 0, 0], 'angle': 12.7}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 12.7}]},
        '41b': {'FCC': [{'axis': [2, 1, 0], 'angle': 40.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 40.9}]},
        '41c': {'FCC': [{'axis': [1, 1, 0], 'angle': 55.9}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 55.9}]},
        '43a': {'FCC': [{'axis': [1, 1, 1], 'angle': 15.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 15.2}]},
        '43b': {'FCC': [{'axis': [2, 1, 0], 'angle': 27.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 27.9}]},
        '43c': {'FCC': [{'axis': [3, 3, 2], 'angle': 60.8}],
                'BCC': [{'axis': [3, 3, 2], 'angle': 60.8}]},
        '45a': {'FCC': [{'axis': [3, 1, 1], 'angle': 28.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 28.6}]},
        '45b': {'FCC': [{'axis': [2, 2, 1], 'angle': 36.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 36.9}]},
        '45c': {'FCC': [{'axis': [2, 2, 1], 'angle': 53.1}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 53.1}]},
        '47a': {'FCC': [{'axis': [3, 3, 1], 'angle': 37.1}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 37.1}]},
        '47b': {'FCC': [{'axis': [3, 2, 0], 'angle': 43.7}],
                'BCC': [{'axis': [3, 2, 0], 'angle': 43.7}]},
        '49a': {'FCC': [{'axis': [1, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 43.6}]},
        '49b': {'FCC': [{'axis': [5, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [5, 1, 1], 'angle': 43.6}]},
        '49c': {'FCC': [{'axis': [3, 2, 2], 'angle': 49.2}],
                'BCC': [{'axis': [3, 2, 2], 'angle': 49.2}]}
    }
    
    # Normalize the given rotation axis
    axis = np.array(axis)
    
    for sigma, configs in sigma_data.items():
        if structure_type not in configs:
            continue
        for config in configs[structure_type]:
            predefined_axis = np.array(config['axis'])
            
            # Check if the given axis is equivalent to the predefined axis
            if are_axes_equivalent(axis, predefined_axis, angle_threshold):
                predefined_angle = config['angle']
                
                # Calculate the deviation
                deviation = abs(misorientation_angle - predefined_angle)
                
                # Calculate the Brandon criterion threshold
                brandon_threshold = 15 / np.sqrt(float(sigma.rstrip('abc')))
                
                # Check if the misorientation angle matches within the Brandon criterion threshold
                if deviation <= brandon_threshold:
                    return True, sigma, deviation
    
    return False, None, None

# Given data
misori_angle = [39.99032809, 34.61751482, 31.8568031, 33.75154261, 47.8394252, 52.18859909, 59.6111677, 45.55147369, 19.51118361, 34.19565999, 28.66987262, 52.3075686, 29.70978892]

Axis = np.array([
    [0.2875, 0.6517, 0.7019],
    [0.4314, 0.0756, 0.899],
    [0.7027, 0.0415, 0.7103],
    [0.1249, 0.0627, 0.9902],
    [0.2368, 0.4618, 0.8548],
    [0.2609, 0.4685, 0.8441],
    [0.5721, 0.5795, 0.5804],
    [0.1318, 0.5213, 0.8432],
    [0.3954, 0.2071, 0.8948],
    [0.0288, 0.1988, 0.9796],
    [0.39, 0.3213, 0.8629],
    [0.6881, 0.1837, 0.702],
    [0.6183, 0.4417, 0.65],
])

# Check each axis and misorientation angle
results = []
for axis, angle in zip(Axis, misori_angle):
    is_sigma, sigma_value, deviation = is_sigma_boundary(axis, angle, structure_type='BCC')
    if is_sigma:
        results.append((axis, angle, sigma_value, deviation))
        print(f"Axis: {axis}, Misorientation Angle: {angle:.2f}°, Σ Boundary: {sigma_value}, Deviation: {deviation:.2f}°")
    else:
        results.append((axis, angle, None, None))
        print(f"Axis: {axis}, Misorientation Angle: {angle:.2f}°, Not a Σ Boundary")



Axis: [0.2875 0.6517 0.7019], Misorientation Angle: 39.99°, Not a Σ Boundary
Axis: [0.4314 0.0756 0.899 ], Misorientation Angle: 34.62°, Σ Boundary: 27b, Deviation: 0.81°
Axis: [0.7027 0.0415 0.7103], Misorientation Angle: 31.86°, Σ Boundary: 27a, Deviation: 0.27°
Axis: [0.1249 0.0627 0.9902], Misorientation Angle: 33.75°, Not a Σ Boundary
Axis: [0.2368 0.4618 0.8548], Misorientation Angle: 47.84°, Not a Σ Boundary
Axis: [0.2609 0.4685 0.8441], Misorientation Angle: 52.19°, Σ Boundary: 39b, Deviation: 2.09°
Axis: [0.5721 0.5795 0.5804], Misorientation Angle: 59.61°, Σ Boundary: 3, Deviation: 0.39°
Axis: [0.1318 0.5213 0.8432], Misorientation Angle: 45.55°, Not a Σ Boundary
Axis: [0.3954 0.2071 0.8948], Misorientation Angle: 19.51°, Not a Σ Boundary
Axis: [0.0288 0.1988 0.9796], Misorientation Angle: 34.20°, Not a Σ Boundary
Axis: [0.39   0.3213 0.8629], Misorientation Angle: 28.67°, Not a Σ Boundary
Axis: [0.6881 0.1837 0.702 ], Misorientation Angle: 52.31°, Σ Boundary: 25b, Deviation:

In [187]:
#Example 3: By two orientations of grains 
#For obtaining rotation axis and misorientation angle
from orix.quaternion import Orientation, symmetry, Rotation
from orix.vector import Vector3d
import numpy as np
from scipy.spatial.transform import Rotation as R

def misorientation_calc(euler_grain1,euler_grain2):
    o1 = Orientation.from_euler(
        np.deg2rad(euler_grain1), symmetry=symmetry.Oh
    )

    o2 = Orientation.from_euler(
        np.deg2rad(euler_grain2), symmetry=symmetry.Oh
    )

    m = o1 - o2

    print(
        "Axis:",
        m.axis,
        "Misori. angle:",
        np.rad2deg(m.angle),
    )
    return  m.axis, np.rad2deg(m.angle)


def are_axes_equivalent(axis2, axis1, angle_threshold=2.0):
    """
    Check if two axes are equivalent considering crystal symmetry and slight deviations by comparing their misorientation angle.
    
    Parameters:
    axis1, axis2 (numpy arrays): The axes to compare
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether the axes are equivalent
    """
    # Normalize both axes
    axis1 = axis1 / np.linalg.norm(axis1)
    axis2 = axis2 / np.linalg.norm(axis2)
    
    # Generate all permutations with sign changes of axis1 to account for symmetry of Cubic system
    axis_permutations = [
        [ axis1[0],  axis1[1],  axis1[2]],
        [-axis1[0],  axis1[1],  axis1[2]],
        [ axis1[0], -axis1[1],  axis1[2]],
        [ axis1[0],  axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1],  axis1[2]],
        [-axis1[0],  axis1[1], -axis1[2]],
        [ axis1[0], -axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1], -axis1[2]],
        [ axis1[1],  axis1[0],  axis1[2]],
        [-axis1[1],  axis1[0],  axis1[2]],
        [ axis1[1], -axis1[0],  axis1[2]],
        [ axis1[1],  axis1[0], -axis1[2]],
        [-axis1[1], -axis1[0],  axis1[2]],
        [-axis1[1],  axis1[0], -axis1[2]],
        [ axis1[1], -axis1[0], -axis1[2]],
        [-axis1[1], -axis1[0], -axis1[2]],
        [ axis1[2],  axis1[1],  axis1[0]],
        [-axis1[2],  axis1[1],  axis1[0]],
        [ axis1[2], -axis1[1],  axis1[0]],
        [ axis1[2],  axis1[1], -axis1[0]],
        [-axis1[2], -axis1[1],  axis1[0]],
        [-axis1[2],  axis1[1], -axis1[0]],
        [ axis1[2], -axis1[1], -axis1[0]],
        [-axis1[2], -axis1[1], -axis1[0]],
        [ axis1[0],  axis1[2],  axis1[1]],
        [-axis1[0],  axis1[2],  axis1[1]],
        [ axis1[0], -axis1[2],  axis1[1]],
        [ axis1[0],  axis1[2], -axis1[1]],
        [-axis1[0], -axis1[2],  axis1[1]],
        [-axis1[0],  axis1[2], -axis1[1]],
        [ axis1[0], -axis1[2], -axis1[1]],
        [-axis1[0], -axis1[2], -axis1[1]],
        [ axis1[1],  axis1[2],  axis1[0]],
        [-axis1[1],  axis1[2],  axis1[0]],
        [ axis1[1], -axis1[2],  axis1[0]],
        [ axis1[1],  axis1[2], -axis1[0]],
        [-axis1[1], -axis1[2],  axis1[0]],
        [-axis1[1],  axis1[2], -axis1[0]],
        [ axis1[1], -axis1[2], -axis1[0]],
        [-axis1[1], -axis1[2], -axis1[0]],
        [ axis1[2],  axis1[0],  axis1[1]],
        [-axis1[2],  axis1[0],  axis1[1]],
        [ axis1[2], -axis1[0],  axis1[1]],
        [ axis1[2],  axis1[0], -axis1[1]],
        [-axis1[2], -axis1[0],  axis1[1]],
        [-axis1[2],  axis1[0], -axis1[1]],
        [ axis1[2], -axis1[0], -axis1[1]],
        [-axis1[2], -axis1[0], -axis1[1]],
    ]
    
    
    # Check if axis2 is equivalent to any of the permutations of axis1 within the specified angle threshold
    for perm in axis_permutations:
        # Calculate the cosine of the angle between the two vectors
        cos_angle = np.dot(perm, axis2)
        
        # Ensure the value is within the valid range for arccos
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        
        # Calculate the angle in degrees
        angle = np.degrees(np.arccos(cos_angle))

        
        # Check if the angle is less than the threshold
        if angle <= angle_threshold or (180 - angle) <= angle_threshold:
            #print(f"Angle between Axis and CSL Axis {perm}: {angle:.2f}°")
            return True, perm
    
    return False, None

def is_sigma_boundary(axis, misorientation_angle, structure_type='FCC', angle_threshold=5.0):
    """
    Determine if the given rotation axis and misorientation angle correspond to a Σ boundary,
    considering the Brandon criterion for deviation and axis symmetry.
    
    Parameters:
    axis (list or tuple): Rotation axis (e.g., [1, 1, 1])
    misorientation_angle (float): Misorientation angle (in degrees)
    structure_type (str): Crystal structure type ('FCC' or 'BCC')
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether it is a Σ boundary
    str: Corresponding Σ value (if it is a Σ boundary)
    float: Actual deviation (in degrees)
    """
    # Predefined Σ values with corresponding rotation axes and misorientation angles for both FCC and BCC
    sigma_data = {
        '3': {'FCC': [{'axis': [1, 1, 1], 'angle': 60}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 60}]},
        '5': {'FCC': [{'axis': [1, 0, 0], 'angle': 36.87}],
              'BCC': [{'axis': [1, 0, 0], 'angle': 36.87}]},
        '7': {'FCC': [{'axis': [1, 1, 1], 'angle': 38.21}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 38.21}]},
        '9': {'FCC': [{'axis': [1, 1, 0], 'angle': 38.94}],
              'BCC': [{'axis': [1, 1, 0], 'angle': 38.94}]},
        '11': {'FCC': [{'axis': [1, 1, 0], 'angle': 50.48}],
               'BCC': [{'axis': [1, 1, 0], 'angle': 50.48}]},
        '13a': {'FCC': [{'axis': [1, 0, 0], 'angle': 22.62}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 22.62}]},
        '13b': {'FCC': [{'axis': [1, 1, 1], 'angle': 27.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 27.79}]},
        '15': {'FCC': [{'axis': [2, 1, 0], 'angle': 48.2}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 48.2}]},        
        '17a': {'FCC': [{'axis': [1, 0, 0], 'angle': 28.07}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 28.07}]},
        '17b': {'FCC': [{'axis': [2, 2, 1], 'angle': 61.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 61.9}]},
        '19a': {'FCC': [{'axis': [1, 1, 0], 'angle': 26.53}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 26.53}]},
        '19b': {'FCC': [{'axis': [1, 1, 1], 'angle': 46.8}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 46.8}]},
        '21a': {'FCC': [{'axis': [1, 1, 1], 'angle': 21.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 21.79}]},
        '21b': {'FCC': [{'axis': [2, 1, 1], 'angle': 44.4}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 44.4}]},
        '23': {'FCC': [{'axis': [3, 1, 1], 'angle': 40.5}],
               'BCC': [{'axis': [3, 1, 1], 'angle': 40.5}]},
        '25a': {'FCC': [{'axis': [1, 0, 0], 'angle': 16.3}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 16.3}]},
        '25b': {'FCC': [{'axis': [3, 3, 1], 'angle': 51.7}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 51.7}]},
        '27a': {'FCC': [{'axis': [1, 1, 0], 'angle': 31.59}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 31.59}]},
        '27b': {'FCC': [{'axis': [2, 1, 0], 'angle': 35.43}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 35.43}]},
        '29a': {'FCC': [{'axis': [1, 0, 0], 'angle': 43.6}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 43.6}]},
        '29b': {'FCC': [{'axis': [2, 2, 1], 'angle': 46.4}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 46.4}]},
        '31a': {'FCC': [{'axis': [1, 1, 1], 'angle': 17.9}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 17.9}]},
        '31b': {'FCC': [{'axis': [2, 1, 1], 'angle': 52.2}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 52.2}]},
        '33a': {'FCC': [{'axis': [1, 1, 0], 'angle': 20.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 20.0}]},
        '33b': {'FCC': [{'axis': [3, 1, 1], 'angle': 33.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 33.6}]},
        '33c': {'FCC': [{'axis': [1, 1, 0], 'angle': 59.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 59.0}]},
        '35a': {'FCC': [{'axis': [2, 1, 1], 'angle': 34.0}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 34.0}]},
        '35b': {'FCC': [{'axis': [3, 3, 1], 'angle': 43.2}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 43.2}]},
        '37a': {'FCC': [{'axis': [1, 0, 0], 'angle': 18.9}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 18.9}]},
        '37b': {'FCC': [{'axis': [3, 1, 0], 'angle': 43.1}],
                'BCC': [{'axis': [3, 1, 0], 'angle': 43.1}]},
        '37c': {'FCC': [{'axis': [1, 1, 1], 'angle': 50.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 50.6}]},
        '39a': {'FCC': [{'axis': [1, 1, 1], 'angle': 32.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 32.2}]},
        '39b': {'FCC': [{'axis': [3, 2, 1], 'angle': 50.1}],
                'BCC': [{'axis': [3, 2, 1], 'angle': 50.1}]},
        '41a': {'FCC': [{'axis': [1, 0, 0], 'angle': 12.7}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 12.7}]},
        '41b': {'FCC': [{'axis': [2, 1, 0], 'angle': 40.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 40.9}]},
        '41c': {'FCC': [{'axis': [1, 1, 0], 'angle': 55.9}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 55.9}]},
        '43a': {'FCC': [{'axis': [1, 1, 1], 'angle': 15.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 15.2}]},
        '43b': {'FCC': [{'axis': [2, 1, 0], 'angle': 27.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 27.9}]},
        '43c': {'FCC': [{'axis': [3, 3, 2], 'angle': 60.8}],
                'BCC': [{'axis': [3, 3, 2], 'angle': 60.8}]},
        '45a': {'FCC': [{'axis': [3, 1, 1], 'angle': 28.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 28.6}]},
        '45b': {'FCC': [{'axis': [2, 2, 1], 'angle': 36.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 36.9}]},
        '45c': {'FCC': [{'axis': [2, 2, 1], 'angle': 53.1}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 53.1}]},
        '47a': {'FCC': [{'axis': [3, 3, 1], 'angle': 37.1}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 37.1}]},
        '47b': {'FCC': [{'axis': [3, 2, 0], 'angle': 43.7}],
                'BCC': [{'axis': [3, 2, 0], 'angle': 43.7}]},
        '49a': {'FCC': [{'axis': [1, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 43.6}]},
        '49b': {'FCC': [{'axis': [5, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [5, 1, 1], 'angle': 43.6}]},
        '49c': {'FCC': [{'axis': [3, 2, 2], 'angle': 49.2}],
                'BCC': [{'axis': [3, 2, 2], 'angle': 49.2}]}
    }
    
    # Normalize the given rotation axis
    axis = np.array(axis)
    
    for sigma, configs in sigma_data.items():
        if structure_type not in configs:
            continue
        for config in configs[structure_type]:
            predefined_axis = np.array(config['axis'])
            
            # Check if the given axis is equivalent to the predefined axis
            if are_axes_equivalent(axis, predefined_axis, angle_threshold)[0]:
                predefined_angle = config['angle']
                standard_axis=are_axes_equivalent(axis, predefined_axis, angle_threshold)[1]

                # Calculate the deviation
                deviation = abs(misorientation_angle - predefined_angle)
                
                # Calculate the Brandon criterion threshold
                brandon_threshold = 15 / np.sqrt(float(sigma.rstrip('abc')))
                
                
                # Check if the misorientation angle matches within the Brandon criterion threshold
                if deviation <= brandon_threshold:
                    return True, sigma, deviation, standard_axis, predefined_angle
    
    return False, None, None, None, None


#euler_grain1 = [78,133.2,232.6]
#euler_grain2 = [275,143.2,256.7]
euler_grain1 = [0,0,0]
euler_grain2=[340.7477,101.71763912,70.74769653]
axis, misorientation_angle =misorientation_calc(euler_grain1,euler_grain2)
crystal_axis = axis[0].data.flatten()

is_sigma, sigma_value, deviation, standard_axis, predefined_angle = is_sigma_boundary(crystal_axis, misorientation_angle[0], structure_type='BCC')
if is_sigma:
    print(f"This is a Σ{sigma_value} boundary with a deviation of {deviation:.2f} degrees")
    print(f"Σ{sigma_value} boundary")
    print(f"standard_axis：{standard_axis}, angle：{predefined_angle:.2f}")
else:
    print("This is not a Σ boundary")




Axis: Vector3d (1,)
[[0.521  0.6036 0.6036]] Misori. angle: [28.22694976]
This is a Σ13b boundary with a deviation of 0.44 degrees
Σ13b boundary
standard_axis：[0.5773502691896258, 0.5773502691896258, 0.5773502691896258], angle：27.79


In [15]:
##Example 4: By two orientations of grains , The Devation calculation is more accurate here
#For obtaining rotation axis and misorientation angle
from orix.quaternion import Orientation, Quaternion, symmetry
from orix.vector import Vector3d
import numpy as np
from scipy.spatial.transform import Rotation as R


def rotation_matrix_to_euler(rotation_matrix):
    """Convert a rotation matrix to Bunge Euler angles."""
    return rotation_matrix.as_euler('ZXZ', degrees=True)


def normalize_vector(vector):
    """Normalize a vector."""
    norm = np.linalg.norm(vector)
    if norm == 0:
        return vector
    return vector / norm


def misorientation_angle(rot1, rot2):
    """Calculate the misorientation angle between two rotation matrices."""
    delta_rot = rot1.inv() * rot2
    angle = delta_rot.magnitude()
    return np.degrees(angle)


def crystal_rotation(euler_angles, axis, angle):
    """Rotate a crystal given initial Euler angles, rotation axis, and rotation angle."""
    # Convert Bunge Euler angles to a rotation matrix
    initial_rotation = R.from_euler('ZXZ', euler_angles, degrees=True)

    # Normalize the rotation axis
    crystal_axis = normalize_vector(axis)

    # Transform the crystal axis to the sample reference frame
    transformed_axis = initial_rotation.apply(crystal_axis)

    # Convert the rotation angle to radians
    angle_radians = np.radians(angle)

    # Create the rotation matrix for the specified axis rotation
    axis_rotation = R.from_rotvec(angle_radians * transformed_axis)

    # Compute the new rotation matrix
    new_rotation = axis_rotation * initial_rotation

    # Get the new Bunge Euler angles
    new_euler_angles = rotation_matrix_to_euler(new_rotation)

    return new_euler_angles


def compute_symmetry_reduced_orientation(ori1, ori2, symmetry_str='Oh'):
    """Compute the symmetry reduced orientations of ori1 with the smallest disorientation angle to ori2."""

    ori1_quat = Quaternion.from_euler(ori1, degrees=True)
    ori1_quat.symmetry = getattr(symmetry, symmetry_str)
    ori2_quat = Quaternion.from_euler(ori2, degrees=True)
    ori2_quat.symmetry = getattr(symmetry, symmetry_str)
    symmetry_ori = ori1_quat.symmetry
    angles = []
    ori2_sym = symmetry_ori.outer(ori2_quat)
    misorientation = ori1_quat * ~ori2_sym
    for orien in misorientation:
        orim = Orientation(orien)
        angles.append(orim.angle)
    angles_array = np.array(angles)
    indices = angles_array.argmin(axis=0)  # Minimum angle down symmetry dim.
    # Index one symmetry orientation for each initial orientation
    out = np.take_along_axis(ori2_sym, indices[np.newaxis], axis=0).squeeze()
    ori2_sym_reduced = out.to_euler(degrees=True)
    return ori2_sym_reduced


def misorientation(rot1, rot2):
    """Calculate the misorientation angle between two rotation matrices."""
    delta_rot = rot1.inv() * rot2
    misorientation_axis = delta_rot.as_rotvec()
    angle = np.degrees(delta_rot.magnitude())
    if 180 - angle < angle:
        misorientation_axis = -misorientation_axis
        angle = 180 - angle
    return misorientation_axis, angle


def misorientation_calc(euler_angles1, euler_angles2):
    """Calculate the misorientation axis and angle between two sets of Euler angles."""
    rot1 = R.from_euler('ZXZ', euler_angles1, degrees=True)
    rot2 = R.from_euler('ZXZ', euler_angles2, degrees=True)
    misorientation_axis, misorientation_ang = misorientation(rot1, rot2)
    return misorientation_axis, misorientation_ang


def misorientation_calc2(euler_grain1, euler_grain2):
    """Calculate the misorientation using orix's Orientation class."""
    o1 = Orientation.from_euler(np.deg2rad(euler_grain1), symmetry=symmetry.Oh)
    o2 = Orientation.from_euler(np.deg2rad(euler_grain2), symmetry=symmetry.Oh)
    m = o1 - o2
    axis = m.axis[0].flatten()
    angle = np.rad2deg(m.angle[0])
    return axis[0].data.flatten(), angle


def are_axes_equivalent(axis2, axis1, angle_threshold=5.0):
    """Check if two axes are equivalent considering crystal symmetry and slight deviations."""
    axis1 = axis1 / np.linalg.norm(axis1)
    axis2 = axis2 / np.linalg.norm(axis2)

    axis_permutations = [
        [ axis1[0],  axis1[1],  axis1[2]], [-axis1[0],  axis1[1],  axis1[2]],
        [ axis1[0], -axis1[1],  axis1[2]], [ axis1[0],  axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1],  axis1[2]], [-axis1[0],  axis1[1], -axis1[2]],
        [ axis1[0], -axis1[1], -axis1[2]], [-axis1[0], -axis1[1], -axis1[2]],
        [ axis1[1],  axis1[0],  axis1[2]], [-axis1[1],  axis1[0],  axis1[2]],
        [ axis1[1], -axis1[0],  axis1[2]], [ axis1[1],  axis1[0], -axis1[2]],
        [-axis1[1], -axis1[0],  axis1[2]], [-axis1[1],  axis1[0], -axis1[2]],
        [ axis1[1], -axis1[0], -axis1[2]], [-axis1[1], -axis1[0], -axis1[2]],
        [ axis1[2],  axis1[1],  axis1[0]], [-axis1[2],  axis1[1],  axis1[0]],
        [ axis1[2], -axis1[1],  axis1[0]], [ axis1[2],  axis1[1], -axis1[0]],
        [-axis1[2], -axis1[1],  axis1[0]], [-axis1[2],  axis1[1], -axis1[0]],
        [ axis1[2], -axis1[1], -axis1[0]], [-axis1[2], -axis1[1], -axis1[0]],
        [ axis1[0],  axis1[2],  axis1[1]], [-axis1[0],  axis1[2],  axis1[1]],
        [ axis1[0], -axis1[2],  axis1[1]], [ axis1[0],  axis1[2], -axis1[1]],
        [-axis1[0], -axis1[2],  axis1[1]], [-axis1[0],  axis1[2], -axis1[1]],
        [ axis1[0], -axis1[2], -axis1[1]], [-axis1[0], -axis1[2], -axis1[1]],
        [ axis1[1],  axis1[2],  axis1[0]], [-axis1[1],  axis1[2],  axis1[0]],
        [ axis1[1], -axis1[2],  axis1[0]], [ axis1[1],  axis1[2], -axis1[0]],
        [-axis1[1], -axis1[2],  axis1[0]], [-axis1[1],  axis1[2], -axis1[0]],
        [ axis1[1], -axis1[2], -axis1[0]], [-axis1[1], -axis1[2], -axis1[0]],
        [ axis1[2],  axis1[0],  axis1[1]], [-axis1[2],  axis1[0],  axis1[1]],
        [ axis1[2], -axis1[0],  axis1[1]], [ axis1[2],  axis1[0], -axis1[1]],
        [-axis1[2], -axis1[0],  axis1[1]], [-axis1[2],  axis1[0], -axis1[1]],
        [ axis1[2], -axis1[0], -axis1[1]], [-axis1[2], -axis1[0], -axis1[1]],
    ]

    for perm in axis_permutations:
        cos_angle = np.dot(perm, axis2)
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        angle = np.degrees(np.arccos(cos_angle))

        if angle <= angle_threshold:
            return True, perm

    return False, None


def is_sigma_boundary(axis, misorientation_angle, structure_type='FCC', angle_threshold=15.0, tolerance_widening=1):
    """Determine if the given rotation axis and misorientation angle correspond to a Σ boundary."""
    sigma_data = {
        '3': {'FCC': [{'axis': [1, 1, 1], 'angle': 60}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 60}]},
        '5': {'FCC': [{'axis': [1, 0, 0], 'angle': 36.87}],
              'BCC': [{'axis': [1, 0, 0], 'angle': 36.87}]},
        '7': {'FCC': [{'axis': [1, 1, 1], 'angle': 38.21}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 38.21}]},
        '9': {'FCC': [{'axis': [1, 1, 0], 'angle': 38.94}],
              'BCC': [{'axis': [1, 1, 0], 'angle': 38.94}]},
        '11': {'FCC': [{'axis': [1, 1, 0], 'angle': 50.48}],
               'BCC': [{'axis': [1, 1, 0], 'angle': 50.48}]},
        '13a': {'FCC': [{'axis': [1, 0, 0], 'angle': 22.62}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 22.62}]},
        '13b': {'FCC': [{'axis': [1, 1, 1], 'angle': 27.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 27.79}]},
        '15': {'FCC': [{'axis': [2, 1, 0], 'angle': 48.2}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 48.2}]},        
        '17a': {'FCC': [{'axis': [1, 0, 0], 'angle': 28.07}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 28.07}]},
        '17b': {'FCC': [{'axis': [2, 2, 1], 'angle': 61.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 61.9}]},
        '19a': {'FCC': [{'axis': [1, 1, 0], 'angle': 26.53}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 26.53}]},
        '19b': {'FCC': [{'axis': [1, 1, 1], 'angle': 46.8}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 46.8}]},
        '21a': {'FCC': [{'axis': [1, 1, 1], 'angle': 21.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 21.79}]},
        '21b': {'FCC': [{'axis': [2, 1, 1], 'angle': 44.4}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 44.4}]},
        '23': {'FCC': [{'axis': [3, 1, 1], 'angle': 40.5}],
               'BCC': [{'axis': [3, 1, 1], 'angle': 40.5}]},
        '25a': {'FCC': [{'axis': [1, 0, 0], 'angle': 16.3}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 16.3}]},
        '25b': {'FCC': [{'axis': [3, 3, 1], 'angle': 51.7}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 51.7}]},
        '27a': {'FCC': [{'axis': [1, 1, 0], 'angle': 31.59}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 31.59}]},
        '27b': {'FCC': [{'axis': [2, 1, 0], 'angle': 35.43}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 35.43}]},
        '29a': {'FCC': [{'axis': [1, 0, 0], 'angle': 43.6}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 43.6}]},
        '29b': {'FCC': [{'axis': [2, 2, 1], 'angle': 46.4}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 46.4}]},
        '31a': {'FCC': [{'axis': [1, 1, 1], 'angle': 17.9}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 17.9}]},
        '31b': {'FCC': [{'axis': [2, 1, 1], 'angle': 52.2}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 52.2}]},
        '33a': {'FCC': [{'axis': [1, 1, 0], 'angle': 20.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 20.0}]},
        '33b': {'FCC': [{'axis': [3, 1, 1], 'angle': 33.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 33.6}]},
        '33c': {'FCC': [{'axis': [1, 1, 0], 'angle': 59.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 59.0}]},
        '35a': {'FCC': [{'axis': [2, 1, 1], 'angle': 34.0}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 34.0}]},
        '35b': {'FCC': [{'axis': [3, 3, 1], 'angle': 43.2}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 43.2}]},
        '37a': {'FCC': [{'axis': [1, 0, 0], 'angle': 18.9}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 18.9}]},
        '37b': {'FCC': [{'axis': [3, 1, 0], 'angle': 43.1}],
                'BCC': [{'axis': [3, 1, 0], 'angle': 43.1}]},
        '37c': {'FCC': [{'axis': [1, 1, 1], 'angle': 50.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 50.6}]},
        '39a': {'FCC': [{'axis': [1, 1, 1], 'angle': 32.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 32.2}]},
        '39b': {'FCC': [{'axis': [3, 2, 1], 'angle': 50.1}],
                'BCC': [{'axis': [3, 2, 1], 'angle': 50.1}]},
        '41a': {'FCC': [{'axis': [1, 0, 0], 'angle': 12.7}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 12.7}]},
        '41b': {'FCC': [{'axis': [2, 1, 0], 'angle': 40.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 40.9}]},
        '41c': {'FCC': [{'axis': [1, 1, 0], 'angle': 55.9}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 55.9}]},
        '43a': {'FCC': [{'axis': [1, 1, 1], 'angle': 15.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 15.2}]},
        '43b': {'FCC': [{'axis': [2, 1, 0], 'angle': 27.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 27.9}]},
        '43c': {'FCC': [{'axis': [3, 3, 2], 'angle': 60.8}],
                'BCC': [{'axis': [3, 3, 2], 'angle': 60.8}]},
        '45a': {'FCC': [{'axis': [3, 1, 1], 'angle': 28.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 28.6}]},
        '45b': {'FCC': [{'axis': [2, 2, 1], 'angle': 36.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 36.9}]},
        '45c': {'FCC': [{'axis': [2, 2, 1], 'angle': 53.1}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 53.1}]},
        '47a': {'FCC': [{'axis': [3, 3, 1], 'angle': 37.1}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 37.1}]},
        '47b': {'FCC': [{'axis': [3, 2, 0], 'angle': 43.7}],
                'BCC': [{'axis': [3, 2, 0], 'angle': 43.7}]},
        '49a': {'FCC': [{'axis': [1, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 43.6}]},
        '49b': {'FCC': [{'axis': [5, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [5, 1, 1], 'angle': 43.6}]},
        '49c': {'FCC': [{'axis': [3, 2, 2], 'angle': 49.2}],
                'BCC': [{'axis': [3, 2, 2], 'angle': 49.2}]}
    }
    
    axis = np.array(axis)

    for sigma, configs in sigma_data.items():
        if structure_type not in configs:
            continue
        for config in configs[structure_type]:
            predefined_axis = np.array(config['axis'])
            if are_axes_equivalent(axis, predefined_axis, angle_threshold)[0]:
                predefined_angle = config['angle']
                standard_axis = are_axes_equivalent(axis, predefined_axis, angle_threshold)[1]
                deviation = abs(misorientation_angle - predefined_angle)
                brandon_threshold = tolerance_widening* 15 / np.sqrt(float(sigma.rstrip('abc')))
                if deviation <= brandon_threshold:
                    return True, sigma, deviation, standard_axis, predefined_angle

    return False, None, None, None, None


# Sample input Euler angles for two grains
euler_grain1 = [274, 143.1, 255.5]
euler_grain2 = [77, 132.9, 231.7]

euler_grain1 = [74.349,25.8538,282.2126]#4
euler_grain2 = [10.5895,15.393, 324.2001]#8

euler_grain1 = [306,37.6,59.7]#5
euler_grain2 = [58.0,30.9,45.0]#6


euler_grain1 = [159.7001, 34.6899, 239.2699]#5
euler_grain2 = [280.9955, 31.0826, 43.0217]#6


# Compute symmetry-reduced orientation
euler_grain2_sym_reduced = compute_symmetry_reduced_orientation(euler_grain1, euler_grain2)
print(f"Symmetry reduced grain2 Euler angle (degrees): {euler_grain2_sym_reduced[0]}")

# Calculate misorientation axis and angle
axis, misorientation_ang = misorientation_calc(euler_grain1, euler_grain2_sym_reduced)
print(f"Rotation axis: {axis[0]}, misorientation angle (degrees): {misorientation_ang}")

# Check if it is a Σ boundary
is_sigma, sigma_value, deviation, standard_axis, predefined_angle = is_sigma_boundary(axis[0], misorientation_ang, structure_type='BCC')
if is_sigma:
    print(f"This is a Σ{sigma_value} boundary")
    print(f"Σ{sigma_value} boundary —— Standard axis: {standard_axis}, standard misorientation angle (degrees): {predefined_angle:.2f}")
    
    # Calculate standard Euler angles for calculating deviation
    std_euler_angles = crystal_rotation(euler_grain1, standard_axis, predefined_angle)
    print(f"Standard grain Bunge Euler angles with CSL Σ{sigma_value} relationship to grain 1 (degrees): {std_euler_angles}")
    
    # Calculate misorientation between perfect CSL orientation and actual grain orientation
    misorientation_axis, misorientation_ang = misorientation_calc(euler_grain2_sym_reduced, std_euler_angles)
    print(f"Deviation to the standard grain with CSL Σ{sigma_value} relationship (degrees): {misorientation_ang}")
else:
    print("This is not a low Σ boundary")


Symmetry reduced grain2 Euler angle (degrees): [148.45273802  67.82449126 247.64301522]
Rotation axis: [-0.11862163  0.58466688  0.01720024], misorientation angle (degrees): [34.19565999]
This is a Σ5 boundary
Σ5 boundary —— Standard axis: [0.0, 1.0, 0.0], standard misorientation angle (degrees): 36.87
Standard grain Bunge Euler angles with CSL Σ5 relationship to grain 1 (degrees): [ 140.47891441   68.63779409 -108.19665367]
Deviation to the standard grain with CSL Σ5 relationship (degrees): [7.54454637]


In [7]:
import tkinter as tk
from tkinter import ttk, messagebox
from orix.quaternion import Quaternion, symmetry
import numpy as np

def compute_symmetry_reduced_orientation(ori1, ori2):
    if not all(isinstance(o, Quaternion) for o in (ori1, ori2)):
        raise TypeError("ori1 and ori2 must be orix.quaternion.Quaternion.")

    if ori2.size != 1 or ori2.ndim != 1:
        raise ValueError("ori2 must have shape (1,).")

    if ori1.symmetry == ori2.symmetry:
        symm = ori1.symmetry
    else:
        symm = ori1.symmetry.outer(ori2.symmetry).unique()
    
    angles = []
    ori1_sym = symm.outer(ori1)
    misorientation = ori2 * ~ori1_sym
    
    for orien in misorientation:
        orim = Quaternion(orien)
        angles.append(orim.angle)
    
    angles_array = np.array(angles)
    indices = angles_array.argmin(axis=0)
    out = np.take_along_axis(ori1_sym, indices[np.newaxis], axis=0).squeeze()
    return ori1.__class__(out)

def euler_to_symmetry_equivalent(euler_angles, symmetry_type):
    final_rotation_quat = Quaternion.from_euler(euler_angles, degrees=True)
    final_rotation_quat.symmetry = getattr(symmetry, symmetry_type)
    
    ori_standard = Quaternion([1, 0, 0, 0])
    ori_standard.symmetry = getattr(symmetry, symmetry_type)
    
    final_rotation = compute_symmetry_reduced_orientation(final_rotation_quat, ori_standard)
    final_euler_angles = final_rotation.to_euler(degrees=True)
    return final_euler_angles

def on_calculate():
    try:
        euler_angles = np.array([float(angle) for angle in euler_entry.get().split()])
        if len(euler_angles) != 3:
            raise ValueError("Please enter three Euler angles separated by spaces.")
        symmetry_type = symmetry_combobox.get()
        equivalent_euler_angles = euler_to_symmetry_equivalent(euler_angles, symmetry_type)
        result_label.config(text=f"Mininum rotation equivalent Euler angles: {equivalent_euler_angles}")
    
        rotation_quat = Quaternion.from_euler(euler_angles, degrees=True)
        rotation_quat.symmetry = getattr(symmetry, symmetry_type)
        symmetry_ori = rotation_quat.symmetry
        ori_sym = symmetry_ori.outer(rotation_quat)
        
        textbox.delete(1.0, tk.END)
        for ori in ori_sym:
            euler_symmetry = ori.to_euler(degrees=True)
            textbox.insert(tk.END, f"{euler_symmetry}\n")
    
    except Exception as e:
        messagebox.showerror("Error", str(e))

# Tkinter setup
root = tk.Tk()
root.title("Euler to Symmetry Equivalent")

mainframe = ttk.Frame(root, padding="10")
mainframe.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

ttk.Label(mainframe, text="Euler Angles (degrees):").grid(row=0, column=0, sticky=tk.E)
euler_entry = ttk.Entry(mainframe)
euler_entry.grid(row=0, column=1)

ttk.Label(mainframe, text="Symmetry Type:").grid(row=1, column=0, sticky=tk.E)

# Dynamically populate the Combobox with symmetry types
labels = [
    s for s in dir(symmetry)
    if isinstance(getattr(symmetry, s), symmetry.Symmetry)
    and not s.startswith("_")
]
symmetry_combobox = ttk.Combobox(mainframe, values=labels, state="readonly")
symmetry_combobox.grid(row=1, column=1)
# Set the default selection to "Oh" if available
if "Oh" in labels:
    symmetry_combobox.set("Oh")
else:
    symmetry_combobox.set(labels[0])
    
calculate_button = ttk.Button(mainframe, text="Calculate", command=on_calculate)
calculate_button.grid(row=2, column=0, columnspan=2, pady=10)

result_label = ttk.Label(mainframe, text="")
result_label.grid(row=3, column=0, columnspan=2)

textbox = tk.Text(mainframe, height=10, width=50)
textbox.grid(row=4, column=0, columnspan=2, pady=10)

root.mainloop()
